In [7]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure routines
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import your CRUD module
from AnimalShelter import AnimalShelter

############################
# Data Manipulation / Model
############################

username = "aacuser"
password = "AccuserPass123!"

db = AnimalShelter(username, password)

# Load ALL documents from MongoDB
df = pd.DataFrame.from_records(db.read({}))

# Drop Mongo _id field
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)

############################
# Dashboard Layout / View
############################

app = JupyterDash(__name__)

# Load and encode logo
image_filename = "Grazioso Salvare Logo.png"
encoded_image = base64.b64encode(open(image_filename, "rb").read()).decode()

# Rescue filter options
rescue_options = [
    {"label": "Water Rescue", "value": "water"},
    {"label": "Mountain or Wilderness Rescue", "value": "mountain"},
    {"label": "Disaster or Individual Tracking", "value": "disaster"},
    {"label": "Reset (All Dogs)", "value": "reset"},
]

# App Layout
app.layout = html.Div([

    # Header
    html.Div([
        html.Img(src=f"data:image/png;base64,{encoded_image}", style={"height": "80px"}),
        html.H4("Grazioso Salvare Search-and-Rescue Dashboard"),
        html.P("Developer: Nicole Hutto (CS-340 Project Two)"),
    ], style={"textAlign": "center"}),

    html.Hr(),

    # Rescue filter radio buttons
    html.Div([
        html.H6("Filter Dogs by Rescue Type:"),
        dcc.RadioItems(
            id="filter-type",
            options=rescue_options,
            value="reset",
            labelStyle={"display": "inline-block", "margin-right": "20px"},
        )
    ], style={"margin": "15px"}),

    html.Hr(),

    # Data table
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": col, "id": col, "selectable": True} for col in df.columns],
        data=df.to_dict("records"),
        page_size=10,
        filter_action="native",
        sort_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left"},
    ),

    html.Br(),
    html.Hr(),

    # Graph + Map side-by-side
    html.Div([
        html.Div(id="graph-id", style={"flex": "1", "padding": "10px"}),
        html.Div(id="map-id", style={"flex": "1", "padding": "10px"}),
    ], style={"display": "flex"}),
])

############################
# Controller / Callbacks
############################


# Build MongoDB query for each rescue type
def build_query(filter_type):
    water_breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
    mountain_breeds = [
        "German Shepherd",
        "Alaskan Malamute",
        "Old English Sheepdog",
        "Siberian Husky",
        "Rottweiler",
    ]
    disaster_breeds = [
        "Doberman Pinscher",
        "German Shepherd",
        "Golden Retriever",
        "Bloodhound",
        "Rottweiler",
    ]

    if filter_type == "water":
        return {
            "animal_type": "Dog",
            "breed": {"$in": water_breeds},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }
    elif filter_type == "mountain":
        return {
            "animal_type": "Dog",
            "breed": {"$in": mountain_breeds},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }
    elif filter_type == "disaster":
        return {
            "animal_type": "Dog",
            "breed": {"$in": disaster_breeds},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
        }
    else:
        return {}  # RESET


# Update table when filter changes
@app.callback(
    Output("datatable-id", "data"),
    Input("filter-type", "value"),
)
def update_table(filter_type):
    query = build_query(filter_type)
    results = pd.DataFrame.from_records(db.read(query))
    if "_id" in results.columns:
        results.drop(columns=["_id"], inplace=True)
    return results.to_dict("records")


# Update pie chart
@app.callback(
    Output("graph-id", "children"),
    Input("datatable-id", "derived_virtual_data")
)
def update_graph(viewData):
    if viewData is None or len(viewData) == 0:
        dff = df.copy()
    else:
        dff = pd.DataFrame(viewData)

    if dff.empty or "breed" not in dff.columns:
        return [html.Div("No data available for this filter.")]

    fig = px.pie(dff, names="breed", title="Breed Distribution for Selected Filter")
    return [dcc.Graph(figure=fig)]


# Highlight selected column in table
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    Input("datatable-id", "selected_columns")
)
def highlight_columns(selected_columns):
    return [
        {"if": {"column_id": col}, "backgroundColor": "#D2F3FF"}
        for col in (selected_columns or [])
    ]


# Update map based on selected row
@app.callback(
    Output("map-id", "children"),
    [
        Input("datatable-id", "derived_virtual_data"),
        Input("datatable-id", "derived_virtual_selected_rows"),
    ],
)
def update_map(viewData, selected_rows):
    if not viewData:
        return []

    dff = pd.DataFrame(viewData)
    idx = selected_rows[0] if selected_rows else 0

    lat = dff.iloc[idx, 13]
    lon = dff.iloc[idx, 14]
    breed = dff.iloc[idx, 4]
    name = dff.iloc[idx, 9]

    return [
        dl.Map(
            center=[30.75, -97.48],
            zoom=10,
            style={"width": "100%", "height": "500px"},
            children=[
                dl.TileLayer(),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup([html.H4(name)])
                    ],
                ),
            ],
        )
    ]


# Run the dashboard
app.run_server(mode="inline")

